# An implementation in the spirit (but not to the letter) of Adger (2017)

In order to get the system of the ground, we need to define a recursive data-type for syntactic objects. Adger uses sets. Equivalently, we can use a tree structure; we want to allow for both binary and unary branching structures.

In [2]:
data SO = Terminal String | Unary SO | Binary SO SO deriving (Show,Eq)

:t (Terminal "John")
:t (Unary (Terminal "John"))
:t (Binary (Terminal "John") (Terminal "John"))

(Terminal "John") :: SO

(Unary (Terminal "John")) :: SO

(Binary (Terminal "John") (Terminal "John")) :: SO

We can think of Adger's "resouce space" as a list of syntactic objects (this is in theory more expressive than a multiset, but this won't matter much for our purposes).

In [3]:
type ResourceSpace = [SO]

We can just treat the lexicon as a list of strings.

In [4]:
type Lexicon = [String]

exampleLex :: Lexicon
exampleLex = ["the","dog","cat","chases","v","T","C"]

Adger's "Operating space" is simply a syntactic object. The "restriction to cardinality of 2" simply falls out from the data-structure we've already defined, since tree structures can maximally be binary branching.

We use `Maybe` here to reflect the fact that the operating space might be empty.

In [5]:
type OperatingSpace = Maybe SO

We can now define some basic operations - "insert" writes an expression from the lexicon into an existing resource space.

N.b. we could make insert "safe" (i.e., not partial) by using `Maybe`, but I won't bother here.

In [6]:
insert :: Lexicon -> Int -> ResourceSpace -> ResourceSpace
insert dict n oldRS = Terminal (dict !! n):oldRS

In [7]:
insert exampleLex 0 []

[Terminal "the"]

In order to define Select, we need a helper function to determine how syntactic objects combine. We may as well call this `merge`. This covers two cases - merging an XP with an empty object returns that XP, and merging an XP with a YP returns a node with XP and YP as immediate daughters.

In [8]:
merge :: SO -> Maybe SO -> SO
merge xp Nothing = xp
merge xp (Just yp) = Binary xp yp

Now we can define the first instance of select (writing from RS to OS) as follows (note that we define a derivational step consisting of a pair of a RS and OS).

In [9]:
type DerivationStep = (ResourceSpace,OperatingSpace)

del :: Int -> [a] -> [a]
del n oldList = (\x -> fst x ++ tail (snd x)) (splitAt n oldList)

select1 :: Int -> DerivationStep -> DerivationStep
select1 n (ds,os) = (del n ds, Just (merge (ds !! n) os))

In [10]:
rs1 = (insert exampleLex 3 . insert exampleLex 2 . insert exampleLex 0) [] 
rs1

[Terminal "chases",Terminal "cat",Terminal "the"]

In [11]:
ds1 = select1 1 (rs1,Nothing)
ds1

([Terminal "chases",Terminal "the"],Just (Terminal "cat"))

In [12]:
ds2 = select1 1 ds1
ds2

([Terminal "chases"],Just (Binary (Terminal "the") (Terminal "cat")))

Select2 is going to be a little more complex - first we need a way of listing every SO that is a subpart of some given SO. Each will be a candidate for selection.

In [13]:
preorderSO :: SO -> [SO]
preorderSO (Terminal x) = [Terminal x]
preorderSO (Unary child) = Unary child : preorderSO child
preorderSO (Binary child1 child2) = (Binary child1 child2:preorderSO child1) ++ preorderSO child2

Let's make sure that this gets everything right.

In [14]:
preorderSO (Terminal "John")

[Terminal "John"]

In [15]:
preorderSO (Unary (Terminal "John"))

[Unary (Terminal "John"),Terminal "John"]

In [16]:
preorderSO (Binary (Terminal "Mary") (Unary (Terminal "swims")))

[Binary (Terminal "Mary") (Unary (Terminal "swims")),Terminal "Mary",Unary (Terminal "swims"),Terminal "swims"]

Now we can define select2.

In [17]:
select2 :: Int -> DerivationStep -> DerivationStep
select2 n (ds,Just os) = (ds, Just (merge (preorderSO os !! n) (Just os)))

Let's see how this gets internal merge.

In [18]:
ds2
select2 2 ds2

([Terminal "chases"],Just (Binary (Terminal "the") (Terminal "cat")))

([Terminal "chases"],Just (Binary (Terminal "cat") (Binary (Terminal "the") (Terminal "cat"))))

Finally, select can write from the operating space to the resource space.

In [24]:
select3 :: Int -> DerivationStep -> DerivationStep
select3 n (ds,Just os) = (xp:ds,remove xp (Just os))
    where xp = preorderSO os !! n
    
-- we need a helper function to remove elements from the resource space.
    
remove :: SO -> OperatingSpace -> OperatingSpace
remove xp Nothing = Nothing
remove xp (Just yp) = if xp == yp then Nothing else Just yp
remove xp (Just (Unary yp)) = if xp == yp then Nothing else Just (Unary yp)
remove xp (Just (Binary yp zp))
    | xp == yp = Just zp
    | xp == zp = Just yp

Now we can show how parallel merge is blocked:

In [25]:
exampleLex2 :: Lexicon
exampleLex2 = ["guess", "what", "anson", "recommended", "before", "anson", "read"]

In [49]:
rs1 = concat $ (insert exampleLex2 <$> [0..6]) <*> [[]]
rs1

[Terminal "guess",Terminal "what",Terminal "anson",Terminal "recommended",Terminal "before",Terminal "anson",Terminal "read"]

In [58]:
ds3 = select1 1 (rs1,Nothing)
ds3
ds4 = select1 0 ds3
ds4
ds5 = select3 0 ds4
ds5
ds6 = select1 0 ds5
ds6

([Terminal "guess",Terminal "anson",Terminal "recommended",Terminal "before",Terminal "anson",Terminal "read"],Just (Terminal "what"))

([Terminal "anson",Terminal "recommended",Terminal "before",Terminal "anson",Terminal "read"],Just (Binary (Terminal "guess") (Terminal "what")))

([Binary (Terminal "guess") (Terminal "what"),Terminal "anson",Terminal "recommended",Terminal "before",Terminal "anson",Terminal "read"],Nothing)

([Terminal "anson",Terminal "recommended",Terminal "before",Terminal "anson",Terminal "read"],Just (Binary (Terminal "guess") (Terminal "what")))

# An alternative architecture using `State`

Idea:
- syntactic objects can be represented as singly linked lists, i.e., trees which have maximally one subtree. Merge is therefore just `(:)`.
- Since syntactic objects are singly linked lists, in order to construct, e.g., a complex specifier, we must build it, "flatten" it via spellout, and push it back into the resource space.
- The derivation is complete when the operation space is empty, and the resource space contains a grammatical sentence.

To keep things maximally simple, let's just terminals as strings.

I've defined a helper function `bracket` to print singly linked lists as bracketed representations.

In [185]:
type Terminal = String

type SO = [Terminal]

merge :: Terminal -> SO -> SO
merge = (:)

mergeM = liftM2 merge

bracket :: SO -> String
bracket [] = ""
bracket (x:xs) = unwords ["[",x,bracket xs,"]"]

In [186]:
bracket ["John","saw","Mary"]

"[ John [ saw [ Mary  ] ] ]"

This means we can keep things *super* simple, and treat both the resource space and operating spaces as...syntactic objects.

In [187]:
import Control.Monad.State
import Data.Functor.Identity

type ResourceSpace = SO
type OperatingSpace = SO

A derivation lives on the State monad - it reads in a syntactic object, i.e., a singly linked list of terminals -- and outputs a syntactic object.

In [188]:
type Derivation a = State SO a

`select` pops a syntactic object off of the resource stack.

In [189]:
select :: Derivation Terminal
select = StateT (\(x:yp) -> Identity (x,yp))

The priveleged starting state is just an empty syntactic object.

In [190]:
start :: Derivation SO
start = return mempty

We can define external merge as `mergeM` applied to `select`. We can now do a simple external merge derivation as follows:

In [191]:
extMerge :: Derivation SO -> Derivation SO
extMerge = mergeM select

example1 = ($ ["John","left"]) . runState $ (extMerge . extMerge) start

(bracket . fst) example1

"[ John [ left  ] ]"

Internal merge doesn't need to make any reference to the global state:

In [192]:
move :: Int -> SO -> SO
move n xp = (xp !! n) `merge` xp

intMerge :: Int -> Derivation SO -> Derivation SO
intMerge n = (<$>) (move n)

We can now do a simple complement movement derivation:

In [193]:
example2 = ($ ["John","likes","who"]) . runState $ (intMerge 2 . extMerge . extMerge . extMerge) start
(bracket . fst) example2

"[ who [ John [ likes [ who  ] ] ] ]"

A problem -- we can't generate structures with complex specifiers/adjuncts!!

The solution -- spell-out pushes a complex structure from the operating space to the back of the queue in the resource space.

In [194]:
spellout :: Derivation SO -> Derivation SO
spellout (StateT der) = StateT $ \rs -> 
    let (os,rs') = runIdentity $ der rs
    in Identity (mempty,unwords os:rs')

A complex specifier derivation + movement:

In [197]:
step1 = ($ ["the","boy","like","who"]) . runState $  spellout $ (extMerge . extMerge) start
step2 = ($ snd step1) . runState $ (intMerge 2 . extMerge . extMerge . extMerge) start

snd step1
(bracket . fst) step2

["the boy","like","who"]

"[ who [ the boy [ like [ who  ] ] ] ]"